In [ ]:
%reset -f

In [ ]:
!pip install GML

In [ ]:
from GML import AutoNLP

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score

import pandas as pd
import numpy as np

In [ ]:
dataset = pd.read_csv('../input/roman-urdu-dataset/Dataset/Roman Urdu DataSet.csv', header=None)

In [ ]:
dataset.drop([2], axis=1, inplace = True)
dataset.columns = ['Text', 'Labels']
dataset.head()

In [ ]:
nlp = AutoNLP()

In [ ]:
nlp.Model_Names()

In [ ]:
MAX_LEN = 100 

nlp.set_params(dataset['Text'].values, tokenizer_name='joeddav/xlm-roberta-large-xnli', 
               model_name='joeddav/xlm-roberta-large-xnli', MAX_LEN=MAX_LEN, epochs = 1, BATCH_SIZE=4)

In [ ]:
def trunc(text):
    text = str(text)
    if len(text) > MAX_LEN:
        return text[:MAX_LEN]
    return text

In [ ]:
dataset['Text'] = dataset['Text'].apply(lambda x: trunc(x))

In [ ]:
%%time

tokenized = nlp.tokenize(dataset['Text'])

In [ ]:
dataset[dataset['Labels'] == 'Neative'] # wrong value
dataset.at[13277, 'Labels'] = 'Negative'

In [ ]:
dataset['Labels'].value_counts()

In [ ]:
le = LabelEncoder()

dataset['Labels'] = le.fit_transform(dataset['Labels'])
dataset['Labels'] = dataset['Labels'].apply(lambda x: int(x))

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(tokenized, dataset['Labels'])

In [ ]:
print(f'{X_train.shape} {y_train.shape}')
print(f'{X_test.shape} {y_test.shape}')

In [ ]:
%%time

model = nlp.train_model(X_train, y_train)

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
accuracy_score(y_test, np.round(y_pred))